In [1]:
import tensorflow as tf
conv_base=tf.keras.applications.MobileNetV2()

In [2]:
conv_base.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [3]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block_10_expand':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [4]:
from keras import models
from keras import layers
from keras.layers import Dropout

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(layers.Dense(4, activation='softmax'))

In [5]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                  horizontal_flip=True, 
                                  rotation_range=10,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=0.01,
                                  zoom_range=0.01)

valid_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        r"D:\fakultet\face-emotion-recognition-main\final_images\train",
        target_size=(640, 480),
        batch_size=16,
        class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(
        r"D:\fakultet\face-emotion-recognition-main\final_images\validation",
        target_size=(640, 480),
        batch_size=16,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        r"D:\fakultet\face-emotion-recognition-main\final_images\test",
        target_size=(640, 480),
        batch_size=16,
        shuffle=False,
        class_mode='categorical')

Found 40878 images belonging to 4 classes.
Found 10037 images belonging to 4 classes.
Found 10308 images belonging to 4 classes.


In [6]:
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.0005)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

history = model.fit(
        train_generator,
        steps_per_epoch=STEP_SIZE_TRAIN,
        validation_data=valid_generator,
        validation_steps=STEP_SIZE_VALID,
        epochs=5
)

Epoch 1/5
2554/2554 [==============================] - 5920s 2s/step - loss: 0.4408 - accuracy: 0.8152 - val_loss: 2.1635 - val_accuracy: 0.4653
Epoch 2/5
2554/2554 [==============================] - 3455s 1s/step - loss: 0.1647 - accuracy: 0.9451 - val_loss: 0.7185 - val_accuracy: 0.7966
Epoch 3/5
2554/2554 [==============================] - 3441s 1s/step - loss: 0.1374 - accuracy: 0.9540 - val_loss: 0.9550 - val_accuracy: 0.7910
Epoch 4/5
2554/2554 [==============================] - 3439s 1s/step - loss: 0.1156 - accuracy: 0.9599 - val_loss: 0.9452 - val_accuracy: 0.7815
Epoch 5/5
2554/2554 [==============================] - 3430s 1s/step - loss: 0.1021 - accuracy: 0.9654 - val_loss: 0.6556 - val_accuracy: 0.8170


In [7]:
test_loss, test_acc = model.evaluate(test_generator)
print('test acc:', test_acc)

645/645 [==============================] - 240s 371ms/step - loss: 1.0891 - accuracy: 0.7891
test acc: 0.7890958189964294


In [8]:
predictions = model.predict(test_generator)
import numpy as np
predicted_class_indices=np.argmax(predictions,axis=1)
correct_classes = np.array(test_generator.classes)
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
M = confusion_matrix(correct_classes, predicted_class_indices)
labels = test_generator.class_indices.keys()
import pandas as pd
print("Test")
pd.DataFrame(M, index=labels, columns=labels)

Test


,eyebrow_raise,frown,smile,squeezed_eyes
eyebrow_raise,2549,42,0,0
frown,115,1828,0,593
smile,122,335,2020,233
squeezed_eyes,58,676,0,1737


In [9]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

C:\Users\Indovski\anaconda3\envs\tf-gpu\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Saved model to disk


In [10]:
filepath = r'D:\fakultet\face-emotion-recognition-main\MobileNetV2_high_intensity_78.9%.h5'
model.save(filepath)